In [15]:
from libs.driverGetter import getDriver
from naver_cafe.parser.cafeBoardSearchParser import parse
import time
import json
from datetime import datetime
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup

driver = getDriver()

url = 'https://www.naver.com/'
#url = 'https://cafe.naver.com/imsanbu'
driver.get(url)

#login (id pw)
driver.find_element_by_xpath('//*[@id="account"]/a').click()
driver.find_element_by_xpath('//*[@id="id"]').send_keys("abn11_11")
driver.find_element_by_xpath('//*[@id="pw"]').send_keys("Shinhan@11")
time.sleep(5)
driver.find_element_by_xpath('//*[@id="log.login"]').click()
#자동로그인방지문자 입력
time.sleep(30)

#타깃 카페로 이동
url = "https://m.cafe.naver.com/remonterrace"
driver.get(url)
time.sleep(3)
#변수 정의 및 검색

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="log.login"]"}
  (Session info: chrome=84.0.4147.89)


In [ ]:
url = "https://m.cafe.naver.com/overseer"
driver.get(url)
time.sleep(3)

In [16]:
#검색어
keyword = "시공 설치 아파트"
driver.find_element_by_xpath('//*[@id="hd"]/div/div[3]/a').click()
driver.find_element_by_xpath('//*[@id="searchQuery"]').send_keys(keyword)
driver.find_element_by_xpath('//*[@id="myCafeSearchForm"]/div/div[2]/a[2]').click()



#base_url = 'https://m.cafe.naver.com/ArticleSearchList.nhn?search.clubid=15724392&search.menuid=0'
test_list = []


#글 url을 불러와서 저장, 20 post / 1 page 
# <ul lst_section > : one page, 20 posts 
#     <li > : post
num_morebtn = 100
for page_num in range(1,num_morebtn):
    
    # 더보기 버튼 50번 클릭
    driver.find_element_by_xpath('//*[@id="moreButton"]').click()
    # 로딩 시간이 있으므로 타이밍 맞추기 위해 sleep(0.5)
    time.sleep(0.8)

In [ ]:
num_morebtn = 50
for page_num in range(1,num_morebtn):
    
    # 더보기 버튼 50번 클릭
    driver.find_element_by_xpath('//*[@id="moreButton"]').click()
    # 로딩 시간이 있으므로 타이밍 맞추기 위해 sleep(0.5)
    time.sleep(1)

In [17]:
#search_list html search result 부분만 추출한 리스트, 인덱스 당 한 페이지(20개 포스트) 
#len(search_list) = num_morebtn
search_html = driver.page_source
search_soup = BeautifulSoup(search_html, 'html.parser')
search_list = search_soup.select('#articleList>ul.lst_section')

In [18]:
post_url=[]
num = 0

#data : one page
#a : <a> 
# id가 bodyContent인 태그의 자손 태그 중에서 a 태그 속성 값 href를 갖은 태그를 찾습니다.
#data.select('#bodyContent a[href]')
for data in search_list: 
    aList=data.select('li > a')
    for a in aList:
        a_url = a.attrs['href']
        post_url.append('https://m.cafe.naver.com' + a_url)
        
        num +=1

In [19]:
print(len(post_url))

2000


In [1]:
#중간에 끊기면 인덱스 찾기
#k=0
#while k < 1000: 
#    print(k, post_url[k])
#    k += 1

#끊긴 곳부터 인덱스수정
#post_url = post_url[504:]

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re #정규표현식
from datetime import datetime

list = []
res_list = []
date_last=""
now = datetime.now()
date_last=str(now.year) + '.'+str(now.month) + '.' + str(now.day)

#print(new_driver.find_elements_by_css_selector('#postContent').text())

def parse(pageString):
    result = ""
    new_line = ""
    bsObj = BeautifulSoup(pageString,'html.parser')
    
    #카페 내 거래게시판 글 걸러내기
    if len(bsObj.select('h4.product_name')) > 0 : 
        #print(bsObj.select('h4.product_name'))
        #print('in IF')
        return None
    
    #카페 등업해야 볼 수 있는 페이지 뜨면 돌아가기
    #if len(bsObj.select('.NHN_Writeform_Main'))== 0 and len(bsObj.select('.post_cont.font_zoom1')) == 0: 
    #    print('Error***********')
    #   return None
    
    #영상부분 text 제거하기
    #source: https://studyforus.com/innisfree/650714
    if len(bsObj.select('.u_rmcplayer.u_rmc_free-resolution')) > 0 : 
        for j in bsObj.select('.u_rmcplayer.u_rmc_free-resolution'): 
            j.decompose()
    
    try:
        title = bsObj.select('h2.tit')[0].get_text()
    except IndexError as err: 
        print('Error***********', err)
        return None
    
    date_temp = bsObj.select('span.date.font_l')[0].get_text()
    date = date_cleansing(date_temp)
    
    if date == None :
        if len(title) >= 1:
            date = "2020.07.27"
            print('date==None, title > 1')
        else:
            print('date==None, title ==0')
            return None
    
    date_last = date
    
    list = bsObj.select('div.post_cont.font_zoom1 > div')
    contents = ''.join([rows.get_text() for rows in list])
    
    if contents == '' or None: 
        return None
    
    title = title.replace("\n","")
    
    for i in contents:
        new_line = new_line + i.replace("\u200b","").replace("\n","").replace("\xa0","") 
    
    new_line.splitlines()
    #res_list.append(new_line)
    
    #res_list.append({'date':date, 'title' : title, 'content' : contents})
    result = date +'\t' + title + new_line + '\n'

    return(result)


#작성일 정제 
def date_cleansing(date):
    pattern ='\d+.(\d+).(\d+)'
    r = re.compile(pattern)
    try:
        match = r.search(date).group(0)
    except AttributeError as err: 
        print('Error***********', err)
        return None
    else:
        return match


In [12]:
#new_driver = getDriver()
count = 0 

file = open('./remonterrace_instl_cnstr.txt','w+',encoding='UTF-8')

for i in post_url: 
    driver.get(i)
    time.sleep(2.0)
    html = driver.page_source 
    soup = parse(html)
    
    if soup == None :
        continue
    
    print(soup+'\n')
    print('**********',count,'************')
    file.write(soup)
    count += 1

file.close()
#driver.close()

2020.07.27	매트나 베이비룸 둘중에 하나는 해야하죠?        전체 매트를 하거나 베이비륨을 하거나 둘중에 하나는 해야하는거죠?이제 슬슬 기어다녀서 거실 매트를 탈출하는데매트 안깔고 키우는 분도 계세요?거실에 베이비룸 설치하기는 싫고.. 어찌해야하나 고민인데안깔고도 살수 있나요 딱딱한 나무 마루인데 ㅜㅜ전세집 나무 마루 상태가 안좋아서 전체 매트 시공하기가 어려우니 고민이네요 ...


********** 0 ************
2020.07.27	[매장소개] 충남 벽걸이티비설치 콘센트연장매립 LED시계시공합니다~        충남 홍성에위치한 충남전지역벽걸이tv설치및셋톱박스숨김,랜선매립,led시계매립,콘센트매립증설전문업체 "TV스카이"입니다^^블로그주소(아래주소클릭!)https://m.blog.naver.com/hsskylife입주아파트만을 겨냥하여저가자재로 반짝시공을 하고사라지는 업체가 아니며,충남권내에서 책임(a/s)시공 하고 있습니다.최근시공사례보세요벽걸이tv설치및 LED시계시공(업체실제시공컷)주방아일랜드식탁및 주방상단콘센트 매립연장시공


********** 1 ************
2020.07.24	깔끔하게 욕실 리모델링 싹 했어요ㅎㅎ        오래된 아파트라 예전부터 욕실 리모델링은 꼭 하고싶었는데 큰맘먹구 드디어 했습니다 ㅎㅎ타일부터 천장, 변기,세면기,수납장,파티션까지 싹 다 갈아치웠더니화장실만보면 남의 집에 와있는 것 같아요 ㅋㅋㅋ리모델링 하기전 화장실이 구식이라서 청소를 깨끗하게해도지저분해보이고공간이 협소해서 한번 샤워하면 변기까지 그냥 다 물난리가나서 불편했는데리모델링하면서 파티션까지 설치했더니 넘나 좋네요^^여러군데 견적봤었는데 다들화장실이 좁고 샤워공간이랑 세면대가 가까워서 파티션설치 어렵다고설치하게되도 엄청 좁아져서 불편할꺼라구 하셨는데이번에 시공하면서 세면대랑 변기 위치 잘 계산해서 옮겨주셔서 샤워공간 더 넓혀주셨어여 ㅠㅠ제 고집때문에 업체에서 많이 힘드셨겠지만...진짜 대만족입니당ㅎㅎ수납장도 작은거라서 욕실용

2020.07.12	[쌍둥이] 쌍둥이67일차 아기침대반납예정. 다들 어떻하시나요?        안녕하세요. 늘 이카페에서 많은 도움 받아이번에도 쌍둥이 맘님들께.조언 여쭈어봅니다^^지금 침실엔.아기침대 2개와 가실엔 꿈비메트 작은것 ( 물려받은갓) 사용중입니다. 사진첨부했어요♡이제대여한 원목침대를 반납해야하는데 보통 오전오후엔 바닥. 저녁은 침대생활하다가지금은 거실매트에서 재우고 거실매트가 작아서저희는 쇼파에서 자거든요. 아예 저희.침실침대에 붙여서 쓰는 패밀리?침대나 븥여쓰는 침대가 있던데 사용해보신분 있으세요?어떤게 좋은지.추찬해주실수 있을까요?아님 거실에 꿈비리코코매트시공해서 거기서 주무시는 분들도 있는거깉던데그거 깔고 가드는 따로 사서 설치하는 건가요?어떤방벚으로 하실때좋으샸는지..시용하신 후기와 추천 부탁드리겠슺니다.오늘도 열육아화이팅입니다. 감사합니다^^ 


********** 15 ************
2020.07.11	[22주] 정말 사소한 부부쌈..        울 남편은 평소 목소리가 크고 말투가 약간 몰아세우고 따지는듯한 말투예요연애합쳐서 5년을 같이 지냈으니 그정도는 제가 알아서 그냥 넘기기도하고 원래 그러려니 하는데 한번씩 남들에게도 그럴땐 옆에있는 제가 짜증이 납니다 (누가보면 따지는줄 알겠다., 왜 말에 짜증을 썪냐..곱게말해라 .).요렇게 이야기하면 막 화내면서 내가 언제 짜증나게 말했냐며 눈을 부라리고 흥분합니다 그럼 전 또 당신이 방금 이야기할때 말투가 그랬다 옆에서 보니 내가 민망했다 이렇게 설명하다 둘 싸움이 되여..남편의 이런점은 어머니랑 제가 너무나 공감하는 점이예요.. 울 남편을 제일 잘아는 어머니도 항상 그것쫌 고쳐라하시거든요 ..근데 의외로 또 애교도많고 행동은 다정다감한데 말투 때문에 기분을 망친답니다요즘 날도 덥고 배도 불러오고 뚱뚱해지고 옷도 진짜 뭘입어도 안이쁘고 목에 쥐젖도 엄청나고해서 짜증도사고 우울한데 아기 생각하며 기분좋게 긍정적으로 생각하며 스스로를 위로해요임신초기에 이사를 하게되서 집 정리도 

2020.07.01	전세집에 무타공 벽걸이티비 설치했어요^^        이번에 이사오면서 기존 티비장은 버리고 벽걸이 티비 설치했어요전세집인데 집 주인이 벽걸이티비 설치는 안된다고 해서 찾아보가가 무타공으로 설치 가능한 업체가 있더라구요스*브로라는 업첸데 아직 홍보가 안되서 모르는 사람들이 많은거 같아요설치하러 와주신 기사님도 너무 친절하시고, 티비 선, 인터넷 공유기 등등 지저분한것들은 싹다 티비 뒤로 가려주셔서 정말 거실이 너무 깔끔해졌네요👍지인들이 놀러왔는데 기존에 있던 저희집 티비 맞냐며, 새로 산거 아니냐고 할 정도로 고급져 졌어요 ㅋㅋ진심 200%만족스런 시공이었습니다!!


********** 26 ************
2020.07.01	에코폼 맘편한매트 구매후기 :)            푸릇푸릇 해 잘드는 예쁜 #거실뷰 지만이제 20개월이 되어 마구 뛰어다니는우리 기쁨이...층간소음 걱정 &혹여나 뛰다가 넘어져서 다칠까 걱정이런저런 걱정을 달고살다가아무래도 아이매트를 깔아야겠다고 결심했어요!!아이의 안전 및 층간소음도 중요하지만예쁜 거실 디자인도 놓칠 수 없어서이곳 저곳 아이매트 검색을 많이 해봤지요~제가 거실매트를 선택하면서 가장 중요하게 고려했던 기준은 3가지였어요!1. 디자인(예쁜 거실 절대 포기못해!!)2. 소재(울 아가가 바닥에 얼굴을 부벼도안전한 소재인지~)3. 표면(다가오는 한여름,살이 매트에 쩍쩍 달라붙는 재질은 싫어요!!)그러다 눈에 뙇 들어온 에코폼 맘편한 매트!   일단 넘나 취향저격인 파스텔색상 어찌 이리 색을 잘 뽑으셨나요...색상이 다 예뻐서어느 색상으로 조합하든 잘어울리겠더라구요하나같이 다 예뻐서 고민고민하다가저희집 거실에 걸린 액자색과 맞춰서 GET!!저는시셀화이트 + 페일스톤 + 세이지카키 + 샌드베이지총 4가지 색상으로 선택했어요주문한 지 이틀 쯤??넘나 커다랗고 넘나 방가운 박스 도착!!   얼른 뜯어보았어요!   두께가 정말 흐뭇합니다이 정도 두께면 층간소음은 물론아이들의 안전도 확실하게 잡아줄 것 같은 

2020.06.22	[기타용품 후기] 무타공 벽걸이        대리석벽 구멍 뚫는게 싫은 저희 부부는 인터넷 검색하다 발견한 스틸브로 무타공 !!!3년이라는 긴 시간동안 많은 연구와 시공을 통해 직접 개발한 방법으로 특허까지 내서 보유중이라고 합니다.큰 티비와 사운드바까지 설치해주시고는 마무리로 직접 설치하신 기사님께서 매달려 안전함을 확인시켜주시는데, 그야말로 어메이징 ~~무려 1800kg의 하중 테스트도 견디는 인증 또한 받았다고 합니다.우리나라에서 유일하게 벽에 손상을 주지않는 벽걸이 브래킷을 사용하고 있다네요.추가로 DB손해보험을 통한 생산물 배상책임보험 1억원과 영업배상책임보험 1억원에 각각 가입되어 있어, 혹여 발생할수 있는 사고에 대한 대비까지 완벽합니다.안에 무드등까지 있어 밤에는 분위기마저 취합니다 ㅋㅋ타공에 대한 고민과 각종 케이블의 너저분함을 한번에 잡아주는 무타공 벽걸이TV설치, 그야말고 강추네요 !!


********** 39 ************
2020.06.22	층간소음으로 허그매트 시공했어요!        코로나로 어린이집 잠깐 쉬던 시절..아기가 친정에 가있었는데시끄럽다는 밑에집의 민원.아기가 없을때 그러는것도 그렇고 다른집 소음까지다짜고짜 그러니 짜증이 나더라구요 ㅋ진짜 소음이 뭔지 보여주고싶었지만 치졸해지니까ㅎㅎ일단 여기저기 알아보고샘플북 먼저 구입했어요근데 이 샘플매트랑 실제매트랑 두께는 차이가 있더라구요(샘플이 더 두꺼운 느낌)거실 시공 사진입니다.좀 어둡게 나왔어요 ㅋ34평인데 복도부터 거실, 주방까지 시공했구요방 문은 싹 닫아놨네요 ㅋ잡다구리한 장난감들을 치우고 청소만 해놓으면큼지막한 에어컨, 거실장, 소파같은건직접 혼자 들고서 시공 잘 해주시더라구요용인 해주시는 담당자분과 기사님이 따로 계신거 같더라구요요즘 코로나때문에 층간소음 민원이 많아져기다리시는 분들이 많다고 하는데담당자분 닥달해서 빨리 해주셨어요(담당자님 죄송하고 감사해요엄청친절하심 감동..)일단 거실 사진입니다.중문이 없다보니 다른집 소음이 밖으로잘 

2020.06.19	[유아용품 후기] 도노도노매트셀프시공했어요          첫째가 매트없는곳만 뛰어다녀서복도에 새로 매트를 살까 아님 시공을 할까 고민하고 있던찰나도노도노에서 셀프시공매트가 런칭되었더라구요금액이 비싸 고민했지만 옳은 선택이였어요 훗일단 너무 예뻐요 말해뭐해요 도노도노인데색상은 연그레이에요 생각보다 밝아서 집이 환해보여요인테리어에 전혀 방해되지 않는 예쁜 연그레이색이에요매트에 냄새도 안나서 바로 깔아서 사용했구요처음에 물티슈로 싹 닦아봤는데 더럽지도 않았어요그리고 타사의 퍼즐매트보다 조금 더 두꺼운것같아요?7갤둘째가 뒤로 콰당해서 안심되는 폭신한 두께구요촉감도 매끄럽고 부들거리면서 끈적거림이 전혀없었어요그리고 퍼즐형이라 폴더매트보다 매트사이에 이물질끼임을 덜할듯해요?설치전모습설치후 모습여자혼자할수있어요다만 땀이 좀 날뿐            ★ 잠깐! 게시글 작성 전, 필독 공지! ★- 카페규정 : http://cafe.naver.com/imsanbu/28123090 - 게시판별 운영 정책 : http://cafe.naver.com/imsanbu/35756864#도노도노매트시공#셀프매트시공#층간소음매트#유아매트#층간소음방지#도노도노TPU매트#시공매트#놀이방매트#도노도노셀프시공매트


********** 50 ************
2020.06.17	중문설치했어요.        중문설치했습니다.깔끔한시공, AS, 재질 너무 맘에 들어 자랑하고 싶습니다.유압식 중문이라 손끼임이 발생하지않고, 열고 닫을때 스르르 자동으로 닫힙니다.아트월과 마루바닥 색상때문에 엄청 고민했는데 사장님과 상의 끝에 흰색 통유리 결정하여 시공했습니다.제가 좀 까다로운 편인데 정말 마음에 들었습니다.저희집 중문 예쁜가요?★ 잠깐! 게시글작성 전, 필독 공지! ★- 카페규정 : http://cafe.naver.com/imsanbu/28123090- 게시판별 운영 정책 : http://cafe.naver.com/imsanbu/35756864


********** 51 ***

...

2007.03.08	우리집 강화마루 시공기        드뎌 거실에 강화마루를 깔았답니다.저번주 토요일날 시공했답니다.before - 저희 아파트는 5년된 아파트라 새아파트라고 할수 있지만 거실은 ^^; 누런 장판이었답니다햇살에 반짝 반짝 빛나고 있는 늠늠한 장판 보이시죠? ^^아무리 거실을 이쁘게 꾸미고 싶어도 정말 인테리어의 마지막이라고 해야하나,,,장판땜에 항상 아쉬운 맘이 들었엇답니다^^;토요일날 시공하시는분이 오셨는데 아이들도 있고하니 나가 있다가 시공 다 끝나면 전화주시겠다고,,,그래서 시공사진이 없답니다^^;아이들과 신랑 모두 산행을 했답니다.비가 올것만 같은 날씨였지만 막내 18개월짜리를 데리고 갓바위를 올랐다는 ㅡ.ㅡ;;거의 제정신이 아니였단 말씀 드리고싶어서리^^;시공이 끝나고 돌아온 집은 정말이지 먼지 먼지~~~돌아온 시간이 오후 세시반정도 됐는데청소마친 시간은 아홉시가 넘었답니다.그리고 담날에도 또 닦고~~~울신랑 덕분에 대청소까지 한다고 고맙다더군요^^;그래도 청소끝나고 나서 발바닥으로 느끼는 느낌은 장판과는 정말 차이가 많이 나더라는~~~강화마루라 긁힘도 없고 정말이지 강추랍니다.게다가 울 작은아이는 아토피가 있어 만약에 본드로 시공하는 마루였다면 망설였겠지만 짜맞춤식 시공이라 냄새하나 없고아이들에게도 아무런 영향을 주지않는다는게 저는 젤 큰 장점이라고 생각해요~~~색상도 울집 몰딩과도 맞아떨어져서 넘 안심~~~마루만 동동뜰까봐 걱정했답니다^^After베란다까지 설치하니 13평 반이 나왔다고 하시더군요~베란다가 더 이뻐보여요~걸레받이도 다시 설치해주시고~~~이렇게 집안분위기가 바뀔지 예상못했답니다~정말 정말 대만족이예요~~~


********** 1153 ************
2007.02.28	펌0)24평 주공아파트 리모델링        집을 사고자 마음먹고 찾아 나선 24평 주공아파트.하지만 정리안된 어수선한 집을 보고 한참을 망설였습니다.집주인이 말하길 10년을 넘게 도배 한번 안하고 고치지두 않던 집이라고.... ㅜㅜ이 말을

2006.11.09	어제 개원한 미즈 산후조리원을 구경하고 왔습니다.        선사 미즈가 롯데백화점(롯데 백화점과 미래 여성병원 사이) 근처로 이전을 했더군요. 그런데, 선사 미즈에 같이 있던 한의원도 함께 옮겼습니다.그런데, 바로 그 한의원에서 산후조리원을 어제 개원했더군요. 산후조리원은 미즈 병원 9층에 있습니다. 한의원은 3층, 4층부터 8층은 미즈 병원입니다.현재는 3분이 입소한 상태입니다. 미즈에서 출산후 개원하기까지 기다리셨다고 하더군요.음~ 첫인상은 우리가 찾던 산후조리원이라는 것입니다. 당연한 이야기지만 늦게 만든 만큼 다른 산후조리원의 장점을 많이 참고한 것 같습니다.홀은 상당히 넓고 햇빛이 잘 들어와 따뜻하고 전망도 좋고...구조도 커피숍 분위기로 만들었더군요.방은 일반실의 경우 다른 산후조리원의 특실 정도로 큰 것 같습니다. 그리고, 개별난방이라 산모가 원하는대로 온도 조절해서 좋더군요. 침대 있고 tv, 냉장고, 탁자, 옷장 등이 있습니다.특실은 대형액정tv와 노트북등이 갖추어져 있더군요. 참고로 특실은 5만원이 추가됩니다.식당,신생아실, 모유 수유실등 모든 시설이 모두 한 층에 있습니다. 더욱이 모두 넓어서 좋은 것 같고요. 방은 28개 있다고 했던 것 같습니다.한의원에서 하는 산후조리원이라 그린 한방과 비슷한 서비스를 하는 것 같습니다. 한약은 2주주고 한의원에서 이루어지는 치료 및 진료는 모두 무료랍니다.(단 3차 진료 이상 갈때는 개인 부담) 산부인과는 바로 아래층에 있다 보니 활용은 가능하지만 산부인과 이용시 별도의 할인은 없는 듯 합니다.같은 건물에 피부실(확실지 않으나 그렇게 들은 것 같음), 산부인과, 한의원들이 있다보니 위급한 상황 발생시 대처가 빠른 것 같습니다. 일주일 프로그램도 보니 미즈 원장님, 한의원장님, 피부과장님, 영양사님 등이 하시는 프로그램이 있더군요. 주말에는 아빠와 하는 프로그램들인데..같이 만들고 운동하고...또 격주로 음악회도 열리고...피부 맛사지는 주 1회, 발 맛사지 주 2회...특히 인상

2006.08.09	오늘..말복이래요 ^o^        아침부터 푹푹 찌는것이... 배부른 예비맘들.. 모두 잠 설치시공.. 출근하셔서.. 넘 피곤하신거 아녜요???오늘은.. 말복이랍니다..... 삼계탕드시공.. 힘내세요..전 오늘 닭이나 먹을수 있을런지 ㅠ.ㅠ아모레도..울 신랑 출근을 시키지말던가 해야징... 출근만하면.. 12시 퇴근이니.. 이일을 우짜면 좋아요 ㅠ.ㅠ어제도 삼겹살에 된장찌개 끓여먹자 해놓구... 야근이라서.. 저혼자 꾸역꾸역 꼬기 꿔먹었드랬지요.. 장봐오고 찌개끓인게 아까워서 -_-;;아침부터.. 삼겹살 꿔서 시골에서 공수해온 무공해 무농약 상추 깻잎에 파채무쳐서.. 아침 거하게 챙겨 먹여서 출근 시켰더니....맘이 뿌듯하긴 하네요 ~헤헤...샤워도 한바탕 했는데.. 왜이리 땀은 삐질삐질 나는것인지...요즘.. 손이랑 발바닥이 부어서.. 걱정이에요... 배도 하루가 다르게 확확 나오구.. 숨은 차서 헥헥거리공... 힘들어요 ㅠ.ㅠ어뭉들.. 낮에 외출하시지 마시공.... 션한 사무실에서 일보시어요.. 나가면.. 쓰러질지도 몰라욤..저는.. 다시 샤워한판하고.. 잠을 청해볼까 합니다...


********** 1161 ************
2006.07.25	별 이상이 없겠죠,,,        그동안 조용히 눈팅만 하던 제가 너무 걱정되어 이렇게 글을 올립니다. 저는 AEG LAVAMAT 세탁기를 약1년 6개월간 사용하고 있습니다,AEG는 국내 다른 세탁기보다 작은 용량과 동급사양이 떨어짐에도 불구하고 당시 고가의 전자제품이였습니다.주위에서 기왕 혼수로 하는 물품이고 한번 사면 적어도 10년 이상을 사용하고 드럼세탁기는 국산보다는 외국에서 먼저 시작된 제품이라 AEG를 선택하는 것이 좋다는 말과 다른 전자제품판매사보다 가격이 조금은 저렴한 홈플러스에서의 직원 말에 의하면 세탁에서 우수하고 서비스도 우수하다는 말에 독일제품의 AEG세탁기를 선택하게 되었습니다.결혼이 2005. 2. 20 가전제품을 신혼집에 2.1에 배달하기로 홈플러스에서 

2006.06.14	동일하이빌- 방출망 공사하시는 아저씨..가끔 오시나요?        동일하이빌 사시면..혹시 가끔씩..방충망 하는 아저씨..오시지 않나요?어디 아파트 가보니..아파트 입구에 방충망 시공--이라고 써놓고 있던데..혹시 동일하이빌에는 그런 설치하시는분 안오시나요?


********** 1165 ************
2006.04.02	방범창 가격은?        많은 분들이 방범창가격을 궁금해 하셔서 당사에서 취급하는 방범창에 대해서 올려드립니다. 아래가격은 시공비 포함가격이며 시공난이도에 따라 기능선택유무에 따라 가격이 변동될수 있음을 다시한번 알려 드립니다.또한 무료로 방문견적이 가능한데 아래내용처럼 소재나 창 크기에 따라 다르기 때문에 견적을 받아보시는 것이 정확하게 계산될 수 있습니다.방범창 선택하는데 조금이나마 도움이 되시길 바랍니다.각각의 제품 이미지 사진을 클릭하시면 전체 안전창종류를 바로가기로 확인하실수 있습니다. <<방범창의 종류>>(1) 함마톤/청자톤(2) 알미늄(3) 스텐레스(4) 주물(모라돈)(5) 티타늄각각의 제품에 대해서 설명해 드리면(1) 함마톤/청자톤 방범창 옛날에 복도창이나 기타 연립주택에 많이 설치되었던 제품입니다.타 제품에 비해 얇고 가늘어서 그만큼 절단하기도 쉬울(?)수 있는 제품입니다. 가격이 저렴해서 고층에 추락방지 안전창으로 쓰시기에는 좋지만 방범효과는 없습니다.( 방송에 많이 탔던 제품이라 인기가-..(ㅠ.ㅠ) ..)시공가격(1m*1m) = 약 40,000 ~ 50,000 선(2) 알미늄 방범창절단면이 깨끗하고 설치했을때도 녹도 안슬고 깔끔한 장점이 있습니다.함마톤보다는 강도는 좋지만 알미늄이라는 소재 특성상 약할수 있습니다. 하지만 보강재를 내장하면 일반 스텐 못지 않게 튼튼합니다. 스텐에 비해 가격이 저렴하고 보강재를 내장하면 방범효과도 우수해서 요즘 많이 설치를 하고 있습니다.(아름다운 방범창 )원파이프 형태의 관통되어 피스로 고정된 격자스타일과 ( 옛날에 많이 쓰여짐) 가로/세로가 사각에 사

2005.05.29	☆준비부터 완성까지, 기저귀 떼기 3단계 비법        하나씩 천천히, 강요하지 않아야 성공하는준비부터 완성까지, 기저귀 떼기 3단계 비법이제 갓 돌을 넘긴 아이에게 배변훈련을 연습시킬 만큼 아이의 빠른 대소변 가리기가 미덕이 된 세상이다. 하지만 기저귀 떼기의 적절한 시기와 원칙을 지키지 않으면, 아이도 엄마도 소득 없이 힘만 들이다 실패로 끝날 가능성이 많다는데…. 기저귀 떼기의 기본 원칙과 단계별 실천 전략을 알아보자.“아이가 두 돌이 넘었는데 아직도기저귀를 차고 다녀요”아이가 또래 아이에 비해 기저귀 떼기가 늦어지는 것 같으면 발달상에 문제가 있는 것은 아닌지, 혹시 머리가 나빠서 그런 것인지 엄마는 고민된다. 하지만 대소변 가리기는 아이의 습관일 뿐 대부분 엄마의 걱정과 거리가 멀다. 모든 발달 과정이 그렇듯 대소변을 가리는 문제 역시 아이들마다 차이가 나기 때문이다.생후 18∼24개월이 적기, 빨리 시작하면 오히려 문제전문가들은 배변훈련의 적정 시기를 생후 18∼24개월이라고 본다. 아이가 방광과 대장을 조절할 수 있고, 엄마가 하는 말도 어느 정도 알아들을 수 있는 시기이기 때문. 또 자신이 느끼는 변의(便意)를 ‘쉬쉬’, ‘응가’ 등의 말로 표현할 수 있는 때이기도 하다. 하지만 생후 18개월이 지난 모든 아이가 하루 아침에 기저귀를 뗄 수 있는 것은 아니다. 다른 성장과정과 마찬가지로 대소변 가리기 역시 개인차가 크다. 옆집 아이와의 괜한 경쟁심리로 무리하게 배변훈련을 일찍 시작했다 야뇨증, 변비 등의 부작용을 보이는 아이도 많다.배변훈련의 성공 키워드는 엄마의 인내심이다너무 어린 시기에, 강압적으로 배변훈련을 실시하는 것은 권할 만한 일은 아니지만, 아이가 스스로 대소변을 가릴 때까지 무작정 기다리는 것도 바람직하지 않다. 적절한 시기에 기저귀 떼는 훈련을 시작하는 것이 아이의 성취감 획득에 긍정적인 영향을 준다. 대소변을 보고 난 후의 뒷처리 같은 문제는 엄마가 특별히 가르쳐 주지 않으면 아이 혼자 알기 어렵다는 것도